In [2]:
import numpy as np
import torch
import math

## Computer quantile for kl_loss_corner

In [8]:
def compute_score(data_path):
    data = np.load(data_path, allow_pickle=True)
    tp = data.item()['tp']
    scores = []
    for i in range(len(tp)):
        prediction = np.array(tp[i][0])
        pred = prediction[:,:8]
        target = np.array(tp[i][1])
        cov = prediction[:,9:]
        score = np.abs(pred-target) / np.sqrt(np.exp(cov))
        for s in score:
            scores.append(list(s))
    return np.array(scores)

def compute_quantile(scores, alpha):
    n = scores.shape[0]
    q_level = np.ceil((n+1)*(1-alpha))/n
    quantile = []
    for i in range(scores.shape[1]):
        q = np.quantile(scores[:,i], q_level, interpolation='higher')
        quantile.append(q)
    return quantile

### disco

In [4]:
data_path = "check/check_loss_two_step_corner_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[5.685563231556715, 5.7333907000666064, 5.8156107496472655, 5.698934531905109, 5.349435233304022, 5.286866601436044, 5.364550922522339, 5.254597136256393]


In [10]:
data_path = "check/check_loss_two_step_corner_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9545
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[17.925578517687292, 16.902509757000026, 17.921683447998383, 16.65845903704566, 15.847321990328444, 13.83974749789184, 15.933211831132338, 13.469367139403102]


In [11]:
data_path = "check/check_loss_two_step_corner_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[48.667429272195506, 47.12830691785341, 47.89193980424878, 42.536190890924544, 42.169949853660526, 46.371733470171094, 41.253330827583945, 41.55867648053199]


### upperbound

In [5]:
data_path = "check/check_loss_two_step_corner_val/upperbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[4.190345185940821, 4.351993435985297, 4.558479310459919, 4.717549387208282, 4.278715029642637, 4.102446480783583, 4.1725601258012475, 3.530299361843318]


In [12]:
data_path = "check/check_loss_two_step_corner_val/upperbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[49.72913250209128, 68.07916849224074, 54.76645263747507, 76.2931006003976, 46.32874805418221, 76.94147233490142, 50.70841262977031, 77.77131089570797]


### lowerbound

In [6]:
data_path = "check/check_loss_two_step_corner_val/lowerbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[6.10302951960435, 6.182331226887726, 5.740435199991636, 6.236356538776135, 6.334263986132204, 5.1828751359733305, 6.399557006521807, 5.073939114533967]


In [13]:
data_path = "check/check_loss_two_step_corner_val/lowerbound/no_rsu/match_all_data_100_5.npy"
confident_range = 0.9973
scores = compute_score(data_path)
quantile = compute_quantile(scores, 1-confident_range)
print(quantile)

[259.92825137969436, 201.07436055964888, 258.2222934053669, 197.05243747520606, 328.5189148613409, 235.0538877505854, 339.6938946129562, 235.97804342045072]


## Computer quantile for kl_loss_center_ind

In [1]:
def compute_score_center(data_path):
    data = np.load(data_path, allow_pickle=True)
    tp = data.item()['tp']
    scores = []
    for i in range(len(tp)):
        prediction = np.array(tp[i][0])
        pred = prediction[:,:8]
        target = np.array(tp[i][1])
        cov = prediction[:,9:]
        std = np.sqrt(np.exp(cov))
        
        score = np.abs(pred-target) / np.sqrt(np.exp(cov))
        for s in score:
            scores.append(list(s))
    return np.array(scores)

def compute_quantile_center(scores, alpha):
    n = scores.shape[0]
    q_level = np.ceil((n+1)*(1-alpha))/n
    quantile = []
    for i in range(scores.shape[1]):
        q = np.quantile(scores[:,i], q_level, interpolation='higher')
        quantile.append(q)
    return quantile

### disco

In [3]:
data_path = "check/check_loss_two_step_center_ind_val/disco/no_rsu/match_all_data_100_5.npy"
confident_range = 0.6827
scores = compute_score_center(data_path)
#quantile = compute_quantile_center(scores, 1-confident_range)
#print(quantile)

[array([[  3.02728224,   1.00175786,   3.02743149,  -1.0021925 ,
         -1.12828481,  -1.0025022 ,  -1.12843406,   1.00144815,
          0.99896538, -11.40015888, -11.5384407 , -10.98087215,
         -9.18057919, -12.28470898, -15.07167435],
       [ 12.76502705,  17.07698631,  14.58918476,  16.79039383,
         13.93546581,  12.62948036,  12.1113081 ,  12.91607285,
          0.99547666,  -8.74861336,  -8.39334106,  -9.50641632,
         -7.2630825 ,  -9.18798447, -11.61131859],
       [  5.58839035, -17.5082531 ,   3.75817919, -17.20705605,
          4.35461426, -13.58284187,   6.18482542, -13.88403893,
          0.99290013,  -8.85629082,  -8.5903635 ,  -8.97162914,
         -7.25579071,  -9.09390736, -11.64298153],
       [ 26.1445694 ,  -2.53610182,  26.37051773,  -0.61354506,
         31.26747894,  -1.18906116,  31.04153061,  -3.11161804,
          0.98091596,  -8.2464838 ,  -8.70400524, -10.02410507,
         -7.06696033, -12.56819344,  -9.95768547],
       [ -9.52821827,   3.3